In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [59]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
train = pd.read_csv("../../../data/real_or_not_nlp/train.csv")

In [4]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [6]:
train.keyword.unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [7]:
from collections import Counter

In [8]:
Counter(train.keyword)

Counter({nan: 61,
         'ablaze': 36,
         'accident': 35,
         'aftershock': 34,
         'airplane%20accident': 35,
         'ambulance': 38,
         'annihilated': 34,
         'annihilation': 29,
         'apocalypse': 32,
         'armageddon': 42,
         'army': 34,
         'arson': 32,
         'arsonist': 34,
         'attack': 36,
         'attacked': 35,
         'avalanche': 30,
         'battle': 26,
         'bioterror': 37,
         'bioterrorism': 30,
         'blaze': 38,
         'blazing': 34,
         'bleeding': 35,
         'blew%20up': 33,
         'blight': 32,
         'blizzard': 37,
         'blood': 35,
         'bloody': 35,
         'blown%20up': 33,
         'body%20bag': 33,
         'body%20bagging': 33,
         'body%20bags': 41,
         'bomb': 34,
         'bombed': 38,
         'bombing': 29,
         'bridge%20collapse': 35,
         'buildings%20burning': 35,
         'buildings%20on%20fire': 33,
         'burned': 33,
         'bu

In [9]:
train.location.unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [10]:
Counter(train.target)

Counter({1: 3271, 0: 4342})

In [11]:
train.text

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [12]:
import re
from string import punctuation
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

english_stopwords = stopwords.words('english')
stemmer = PorterStemmer()

def transform_tweet(text):
    text = re.sub("#", "", text)
    text = re.sub("\@[a-zA-Z_-]+", "", text)
    text = re.sub("^\s+", "", text)
    text = re.sub("\s+$", "", text)
    text = re.sub(f'[{punctuation}]', "", text)
    
    tokenizer = TweetTokenizer()
    words = tokenizer.tokenize(text)
    words = [word for word in words if word not in english_stopwords]
    words = [stemmer.stem(word) for word in words]
    
    return words

In [13]:
transform_tweet(train.iloc[10].text)

['three', 'peopl', 'die', 'heat', 'wave', 'far']

In [14]:
transformed_text = [transform_tweet(text) for text in train.text]

In [16]:
from tqdm import tqdm

In [44]:
freqs = {}
for index, sentence in tqdm(train.iterrows()):
    for word in transformed_text[index]:
        if sentence.target == 1:
            freqs[(word, 1.0)] = freqs.get((word, 1.0), 0) + 1
        else:
            freqs[(word, 0.0)] = freqs.get((word, 1.0), 0) + 1

7613it [00:02, 3200.44it/s]


In [45]:
freqs

{('our', 1.0): 16,
 ('deed', 1.0): 1,
 ('reason', 1.0): 8,
 ('earthquak', 1.0): 47,
 ('may', 1.0): 50,
 ('allah', 1.0): 6,
 ('forgiv', 1.0): 1,
 ('us', 1.0): 14,
 ('forest', 1.0): 50,
 ('fire', 1.0): 270,
 ('near', 1.0): 49,
 ('La', 1.0): 11,
 ('rong', 1.0): 1,
 ('sask', 1.0): 1,
 ('canada', 1.0): 10,
 ('all', 1.0): 22,
 ('resid', 1.0): 9,
 ('ask', 1.0): 5,
 ('shelter', 1.0): 6,
 ('place', 1.0): 17,
 ('notifi', 1.0): 1,
 ('offic', 1.0): 41,
 ('No', 1.0): 34,
 ('evacu', 1.0): 101,
 ('order', 1.0): 24,
 ('expect', 1.0): 22,
 ('13000', 1.0): 4,
 ('peopl', 1.0): 106,
 ('receiv', 1.0): 4,
 ('wildfir', 1.0): 80,
 ('california', 1.0): 115,
 ('just', 1.0): 19,
 ('got', 1.0): 29,
 ('sent', 1.0): 4,
 ('photo', 1.0): 28,
 ('rubi', 1.0): 1,
 ('alaska', 1.0): 5,
 ('smoke', 1.0): 14,
 ('pour', 1.0): 2,
 ('school', 1.0): 32,
 ('rockyfir', 1.0): 4,
 ('updat', 1.0): 36,
 ('hwi', 1.0): 10,
 ('20', 1.0): 14,
 ('close', 1.0): 29,
 ('direct', 1.0): 6,
 ('due', 1.0): 24,
 ('lake', 1.0): 9,
 ('counti', 1.0):

In [46]:
# EVALUATION

In [47]:
test = pd.read_csv("../../../data/real_or_not_nlp/test.csv")

In [48]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [49]:
test.describe()

,id
count,3263.000000
mean,5427.152927
std,3146.427221
min,0.000000
25%,2683.000000
50%,5500.000000
75%,8176.000000
max,10875.000000


In [50]:
# for _, row in test.iterrows():
#     transformed_tweet = transform_tweet(row.text)
#     print(transformed_tweet)

In [51]:
logprior = np.log(sum(train.target == 1)) - np.log(sum(train.target == 0))
V = len(train)
N_pos = sum(train.target == 1)
N_neg = sum(train.target == 0)

In [52]:
print(logprior)
print(V)
print(N_pos)
print(N_neg)

-0.28323932289985443
7613
3271
4342


In [57]:
def evaluate_tweet(transformed_tweet):
    result = logprior
    
    for word in transformed_tweet:
        freq_pos = freqs.get((word, 1.0), 0.0)
        freq_neg = freqs.get((word, 0.0), 0.0)
        
        result += (np.log((freq_pos + 1.0) / float(N_pos + V)) - np.log((freq_neg + 1.0) / float(N_neg + V)))
        
    return result

In [71]:
#evaluate_tweet(transform_tweet(test.iloc[2]['text']))
results = []
for index, tweet in tqdm(test.iterrows()):
    print(tweet.text)
    result = evaluate_tweet(transform_tweet(tweet.text))
    print(result)
    results.append(int(result >= 0))
    
test['target'] = results

184it [00:00, 847.17it/s]

Just happened a terrible car crash
0.20508546118490756
Heard about #earthquake is different cities, stay safe everyone.
0.1524611543602763
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
3.70929625257574
Apocalypse lighting. #Spokane #wildfires
0.8935533806385108
Typhoon Soudelor kills 28 in China and Taiwan
3.5172427461571463
We're shaking...It's an earthquake
0.08533939405764013
They'd probably still show more life than Arsenal did yesterday, eh? EH?
-0.012019979800116154
Hey! How are you?
-0.35257286591116976
What a nice hat?
-0.5902753077406651
Fuck off!
-0.12039067142361404
No I don't like cold!
0.09965696607298336
NOOOOOOOOO! Don't do that!
-0.11494584877837877
No don't tell me that!
0.10027078821517676
What if?!
-0.2183710797838181
Awesome!
-0.5948486510187312
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
1.1880857563936265
@sunkxssedharry will you wear sho

356it [00:00, 850.87it/s]

-0.14431792709070912
mo the way she says 'carry' https://t.co/vQzRUTHRNU
-0.2501971897283317
Arsonist Sets NYC Vegetarian Restaurant on Fire: Police #NewYork - http://t.co/agn4cL4uSK
2.2796860031714252
Arsonist torches a house himself and his getaway vehicle police say - Surrey Leader http://t.co/q0J5lTVD6k @RichardGEarl
1.1550990295435302
@Safyuan i already threw away the letter tho
-0.358072024352869
Hotel arsonist who caused å£70000 damage was Û÷crying for helpÛª: An arsonist has been jailed after a court hear... http://t.co/HNGDdBhewa
1.6222631628623474
suspected serial arsonist in east bay under arrest @christinKPIX5 @NightBeatTV 10pm @KBCWtv @VeronicaDLCruz
0.6890319595709613
ARSONIST Mc - CHRIST WALK http://t.co/D8LQHiGjT0 #Toronto #Columbus #Gospel
0.04366840392165816
@Gay4BB bitch dont get slapped
-1.2196728699304042
@trap_vodka my new fave vine
-0.9560096204967401
Suspected serial arsonist arrested in NorCal http://t.co/Dty4t75CGu http://t.co/MjxbygejyL
1.1413360117172973
S

564it [00:00, 938.69it/s]

...

-0.3965621392575187
POLLS:  SHOULD THE UNBORN BABIES OF BOKO HARAM TERRORIST BE KILLED IN THEIR MOTHERS WOMB OR SPARED?                http://t.co/3skw0DqZ0r
0.9749827008714376
RT AbbsWinston: #Zionist #Terrorist kidnapped 15 #Palestinians in overnight terror on Palestinian Villages Û_ http://t.co/J5mKcbKcov
20.420901538316972
Reading rec today: Faisal Devji's Terrorist in Search of Humanity on extremists using idea ever since the bomb of humanity as single entity
3.454505141886398
@VICE no sympathyONLF is a terrorist group affiliated with AL shabab This is war! Go report on  war crimes committed by Western powers!
2.373781144063755
RT AbbsWinston: #Zionist #Terrorist Demolish Tire Repair Shop Structure in #Bethlehem
http://t.co/ph2xLI8nVe http://t.co/22fuxHn7El
9.37895101498681
Tltltltltlttlt @P_Grolsh: Ladies are doing gymnastics in the shower!!! Opening legs wider wololo #MetrofmTalk'
-1.4585554242052945
To whom we shld ask tht from where this bldy PAK terrorist has entered in our 

3128it [00:03, 804.80it/s]

-0.5547158432685024
@DArchambau THX for your great encouragement and for RT of NEW VIDEO http://t.co/cybKsXHF7d The Coming Apocalyptic US Earthquake &amp; Tsunami
0.3889149026729357
When tsunami says your order will take 40 minutes and you placed your order when you were on your way ????
2.2295746766287623
She kept it wet like tsunami she think she saw the titanic I get it hot like wasabi
-0.3035822235907415
Oh itÛªs a soccer ball? I thought it was the head of a melting snowman.
-0.6007732524517024
I liked a @YouTube video http://t.co/18aAsFMVbl Call Of Duty Ghosts Walktrough Part-8 Tsunami!
0.9744814102247861
I think Tsunami has hit the wall. Dang... #STLCards
0.3448782593276376
@TA7LPlays @Twitch_Bang @Maliceqt @Zelse007 @RhyareeGaming @dougiedabandit sorry what you say theres a tweet tsunami happening
0.5345994271135899
This high-tech sphere can be the key to saving lives during a tsunami. What would you pack in it? http://t.co/KwlpFqRBVp
1.0008804244394858
Drunk #BBMeg! ??????
-0.

3263it [00:03, 921.05it/s]

2.4432377103396132
GLORIA JONES - VIXEN / WINDSTORM (2IN1) CD 20 TRACKS MARC BOLAN http://t.co/E3cAymmIcN http://t.co/eRzcXloPDL
0.42427607140140555
@t_litt44 I seen you that one day look like you went threw a windstorm ??
0.37149510400531005
Great time group camping at Presqu'ile with family&amp;friends.Lots of fun;not much sleep #PQPHistoryWeekend #windstorm http://t.co/0tYkaZyQpF
0.4751751047273789
Texas Seeks Comment on Rules for Changes to Windstorm Insurer: The Texas Department of Insurance is seeking pu... http://t.co/NGF9n8Jquu
0.45865865539625394
A few more photos from this mornings windstorm. We've never seen anything like it! #rhodeisland #warwick
1.2821845572604094
Police Officer Wounded Suspect Dead After Exchanging Shots http://t.co/e4wvXQItV4
4.869873560418122
The South Korean army wounded 44 persons' Takeshima's Japanese fisherman and occupies the island.  
http://t.co/mJCXgKU8Yt
2.7599965837710942
Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang me

In [72]:
sum(test.results == 1)

2125

In [73]:
sum(test.results == 0)

1138

In [74]:
test

,id,keyword,location,text,results,target
0,0,NaN,NaN,Just happened a terrible car crash,1,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1,1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1,1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1,1
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1,1
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1,1


In [76]:
test.to_csv('submission_naive_bayes1.csv', columns=['id', 'target'], index=False)